In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np
import librosa
import os
from sklearn.model_selection import train_test_split

# Dataset tùy chỉnh cho dữ liệu âm thanh
class AudioDataset(Dataset):
    def __init__(self, file_paths, labels, transform=None):
        self.file_paths = file_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        file_path = self.file_paths[idx]
        label = self.labels[idx]
        # Load âm thanh và trích xuất đặc trưng MFCC
        audio, sr = librosa.load(file_path, sr=None)
        mfcc = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=40)
        mfcc = np.expand_dims(mfcc, axis=0)  # Thêm chiều cho kênh
        if self.transform:
            mfcc = self.transform(mfcc)
        return torch.tensor(mfcc, dtype=torch.float32), torch.tensor(label, dtype=torch.long)

# Thiết kế mô hình CNN bằng PyTorch
class AudioCNN(nn.Module):
    def __init__(self):
        super(AudioCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = None  # Sẽ được khởi tạo sau
        self.fc2 = nn.Linear(128, 4)  # 4 lớp âm thanh

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.view(x.size(0), -1)  # Điều chỉnh kích thước để phù hợp với fully connected layer
        # self.fc1 = nn.Linear(32 * 10 * 107, 128)
        if self.fc1 is None:
            self.fc1 = nn.Linear(x.size(1), 128).to(x.device)  # Khởi tạo fc1 với kích thước phù hợp
        x = F.relu(self.fc1(x))
        x = self.fc2(x)  # Bỏ softmax ở đây vì CrossEntropyLoss đã bao gồm
        return x

# Tạo mô hình
model = AudioCNN()

# Compile mô hình (PyTorch sử dụng cách huấn luyện trực tiếp thay vì compile như Keras)
optimizer = optim.Adam(model.parameters(), lr=0.0001)  # Giảm learning rate để huấn luyện chậm hơn
criterion = nn.CrossEntropyLoss()

# Lấy danh sách tất cả các file audio trong thư mục
audio_dir = r"E:\dataset\sound-data\sound-data\audio"
file_paths = [os.path.join(audio_dir, file) for file in os.listdir(audio_dir) if file.endswith(".wav")]

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
train_paths, test_paths, train_labels, test_labels = train_test_split(file_paths, [0 for _ in range(len(file_paths))], test_size=0.3, random_state=42)  # Tăng tỷ lệ dữ liệu kiểm tra

# Tạo dataset và dataloader cho tập huấn luyện
train_dataset = AudioDataset(train_paths, train_labels)
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)

test_dataset = AudioDataset(test_paths, test_labels)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)

# Tải mô hình nếu đã có mô hình được lưu trước đó
if os.path.exists("best_audio_classification_model.pth"):
    model.load_state_dict(torch.load("best_audio_classification_model.pth"), strict=False)
    print("Đã tải mô hình từ file 'best_audio_classification_model.pth'")

# Huấn luyện mô hình
model.train()
epoch = 0
best_accuracy = 0.0
while True:  # Huấn luyện tới khi đạt độ chính xác mong muốn
    running_loss = 0.0
    for _ in range(5):  # Huấn luyện ít nhất 5 lần mỗi epoch
        for inputs, targets in train_loader:
            try:
                optimizer.zero_grad()
                # Forward
                outputs = model(inputs)
                # Tính loss
                loss = criterion(outputs, targets)
                # Backward
                loss.backward()
                # Cập nhật trọng số
                optimizer.step()
                # Cộng dồn loss
                running_loss += loss.item()
            except RuntimeError as e:
                print(f"Error during training: {e}")
    print(f"Epoch {epoch+1}, Loss: {running_loss / len(train_loader)}")
    
    # Kiểm tra độ chính xác trên tập kiểm tra
    correct = 0
    total = 0
    model.eval()
    with torch.no_grad():
        for inputs, targets in test_loader:
            try:
                outputs = model(inputs)
                predicted = torch.argmax(outputs, dim=1)
                total += targets.size(0)
                correct += (predicted == targets).sum().item()
            except RuntimeError as e:
                print(f"Error during evaluation: {e}")
    accuracy = 100 * correct / total
    print(f"Độ chính xác trên tập kiểm tra: {accuracy}%")
    model.train()
    
    # Lưu mô hình nếu đạt độ chính xác tốt hơn
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        torch.save(model.state_dict(), "best_audio_classification_model.pth")
        print("Đã lưu mô hình tốt nhất vào file 'best_audio_classification_model.pth'")
    
    if accuracy >= 100.0:
        break
    epoch += 1

print("Huấn luyện hoàn tất!")

# Lưu trạng thái tối ưu sau khi huấn luyện
torch.save(model.state_dict(), "final_audio_classification_model.pth")
print("Đã lưu mô hình cuối cùng vào file 'final_audio_classification_model.pth'")

# Đánh giá mô hình
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, targets in test_loader:
        try:
            outputs = model(inputs)
            predicted = torch.argmax(outputs, dim=1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()
        except RuntimeError as e:
            print(f"Error during evaluation: {e}")

print(f"Độ chính xác của mô hình: {100 * correct / total}%")

# Dự đoán cho một file duy nhất
single_loader = DataLoader(train_dataset, batch_size=1, shuffle=False)
model.eval()
with torch.no_grad():
    for inputs, targets in single_loader:
        try:
            outputs = model(inputs)
            predicted = torch.argmax(outputs, dim=1)
            print(f"Predicted: {predicted.item()}, Actual: {targets.item()}")
        except RuntimeError as e:
            print(f"Error during prediction: {e}")

print("Dự đoán hoàn tất!")

# Tiếp tục huấn luyện từ mô hình đã lưu
model.train()
epoch = 0
while True:  # Tiếp tục huấn luyện tới khi đạt độ chính xác mong muốn
    running_loss = 0.0
    for _ in range(5):  # Huấn luyện ít nhất 5 lần mỗi epoch
        for inputs, targets in train_loader:
            try:
                optimizer.zero_grad()
                # Forward
                outputs = model(inputs)
                # Tính loss
                loss = criterion(outputs, targets)
                # Backward
                loss.backward()
                # Cập nhật trọng số
                optimizer.step()
                # Cộng dồn loss
                running_loss += loss.item()
            except RuntimeError as e:
                print(f"Error during training: {e}")
    print(f"Epoch {epoch+1}, Loss: {running_loss / len(train_loader)}")
    
    # Kiểm tra độ chính xác trên tập kiểm tra
    correct = 0
    total = 0
    model.eval()
    with torch.no_grad():
        for inputs, targets in test_loader:
            try:
                outputs = model(inputs)
                predicted = torch.argmax(outputs, dim=1)
                total += targets.size(0)
                correct += (predicted == targets).sum().item()
            except RuntimeError as e:
                print(f"Error during evaluation: {e}")
    accuracy = 100 * correct / total
    print(f"Độ chính xác trên tập kiểm tra: {accuracy}%")
    model.train()
